In [1]:
import joblib
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR:", DATA_DIR)
print("MODELS_DIR:", MODELS_DIR)

PROJECT_ROOT: /opt/app-root/src/sammo-fight-iq
DATA_DIR: /opt/app-root/src/sammo-fight-iq/data
MODELS_DIR: /opt/app-root/src/sammo-fight-iq/models


In [2]:
danger_model = joblib.load(MODELS_DIR / "danger_predictor.joblib")
focus_model  = joblib.load(MODELS_DIR / "focus_predictor.joblib")
feature_cols = joblib.load(MODELS_DIR / "feature_columns.pkl")

print("Loaded models and features:")
print("feature_cols:", feature_cols)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [3]:
from pathlib import Path
import joblib
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR:", DATA_DIR)
print("MODELS_DIR:", MODELS_DIR)

PROJECT_ROOT: /opt/app-root/src/sammo-fight-iq
DATA_DIR: /opt/app-root/src/sammo-fight-iq/data
MODELS_DIR: /opt/app-root/src/sammo-fight-iq/models


In [4]:
danger_model = joblib.load(MODELS_DIR / "danger_predictor.joblib")
focus_model  = joblib.load(MODELS_DIR / "focus_predictor.joblib")
feature_cols = joblib.load(MODELS_DIR / "feature_columns.pkl")

print("Loaded models and feature columns:")
print("feature_cols:", feature_cols)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [5]:
# src/risk_model.py
def video_form_and_danger(stats: dict) -> dict:
    guard_down = stats["guard_down_ratio"]
    pose_cov   = stats["pose_coverage"]

    danger = 0.6 * guard_down + 0.4 * (1.0 - pose_cov)
    danger = max(0.0, min(1.0, danger))

    form = 10.0
    form -= guard_down * 5.0
    form -= (1.0 - pose_cov) * 2.0
    form = max(0.0, min(10.0, form))

    if danger >= 0.7:
        focus = "defense_first"
    elif danger >= 0.4:
        focus = "ring_cutting"
    else:
        focus = "pressure_and_body"

    out = dict(stats)
    out["video_danger_score"] = float(danger)
    out["video_form_score"] = float(form)
    out["video_focus_next_round"] = focus
    return out

In [6]:
pip install memlayer sentence-transformers fastapi uvicorn requests

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 78.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 191.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 129.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 138.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 103.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 120.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 110.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 218.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 213.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
# Create src/simple_memory.py directly in notebook
from pathlib import Path

PROJECT_ROOT = Path("/opt/app-root/src/sammo-fight-iq")
PROJECT_ROOT.mkdir(exist_ok=True)
(PROJECT_ROOT / "src").mkdir(exist_ok=True)

simple_memory_code = '''import json
from pathlib import Path
from typing import List, Dict

class SimpleMemoryStore:
    """Ultra-simple persistent memory using JSONL."""
    
    def __init__(self, path: str = "mem_store.jsonl"):
        self.path = Path(path)
        self.path.parent.mkdir(parents=True, exist_ok=True)
        if not self.path.exists():
            self.path.write_text("")
    
    def append(self, agent_id: str, user_id: str, content: str):
        record = {"agent_id": agent_id, "user_id": user_id, "content": content}
        with self.path.open("a", encoding="utf-8") as f:
            f.write(json.dumps(record) + "\\n")
    
    def get_recent(self, agent_id: str, user_id: str, k: int = 5) -> List[str]:
        if not self.path.exists():
            return []
        
        lines = []
        with self.path.open("r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        lines.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
        
        filtered = [
            rec["content"] for rec in lines
            if rec.get("agent_id") == agent_id and rec.get("user_id") == user_id
        ]
        return filtered[-k:]
'''

(PROJECT_ROOT / "src" / "simple_memory.py").write_text(simple_memory_code)
print("✅ Created src/simple_memory.py")

✅ Created src/simple_memory.py


In [9]:
llm_client_code = '''from dataclasses import dataclass
from typing import List, Dict
import requests
import os

@dataclass
class LLMConfig:
    base_url: str
    model: str
    timeout: int = 60
    temperature: float = 0.3

class LocalLLMClient:
    """OpenAI-compatible client for local LLMs."""
    
    def __init__(self, config: LLMConfig):
        self.config = config
    
    def complete(self, messages: List[Dict[str, str]]) -> str:
        payload = {
            "model": self.config.model,
            "messages": messages,
            "temperature": self.config.temperature,
            "max_tokens": 1024,
        }
        
        try:
            resp = requests.post(self.config.base_url, json=payload, timeout=self.config.timeout)
            resp.raise_for_status()
            data = resp.json()
            
            if "choices" in data:
                return data["choices"][0]["message"]["content"]
            elif "response" in data:
                return data["response"]
            else:
                raise ValueError(f"Unexpected response: {data}")
        except requests.exceptions.RequestException as e:
            return f"[LLM Error: {str(e)}]"

def get_llm_config() -> LLMConfig:
    return LLMConfig(
        base_url=os.getenv("LLM_BASE_URL", "http://localhost:11434/v1/chat/completions"),
        model=os.getenv("LLM_MODEL", "mistral:7b-instruct"),
    )
'''

(PROJECT_ROOT / "src" / "llm_client.py").write_text(llm_client_code)
print("✅ Created src/llm_client.py")

✅ Created src/llm_client.py


In [10]:
memory_layer_code = '''from typing import Optional, Dict
import json
from .llm_client import LocalLLMClient
from .simple_memory import SimpleMemoryStore

class MemoryBackedLLM:
    """LLM with persistent conversation memory."""
    
    def __init__(self, llm_client: LocalLLMClient, mem_path: str = "mem_data/mem_store.jsonl"):
        self.llm = llm_client
        self.store = SimpleMemoryStore(mem_path)
    
    def chat(self, agent_id: str, user_id: str, system_prompt: str, 
             message: str, context_data: Optional[Dict] = None) -> str:
        
        # Get recent history
        history_chunks = self.store.get_recent(agent_id, user_id, k=5)
        history_text = "\\n".join(history_chunks)
        
        # Build messages
        messages = [{"role": "system", "content": system_prompt}]
        
        if history_text:
            messages.append({"role": "system", "content": f"Recent history:\\n{history_text}"})
        
        if context_data:
            messages.append({"role": "system", "content": f"Current stats:\\n{json.dumps(context_data, indent=2)}"})
        
        messages.append({"role": "user", "content": message})
        
        # Get response
        reply = self.llm.complete(messages)
        
        # Store interaction
        self.store.append(agent_id, user_id, f"User: {message}\\nCoach: {reply}")
        
        return reply
'''

(PROJECT_ROOT / "src" / "memory_layer.py").write_text(memory_layer_code)
print("✅ Created src/memory_layer.py")

✅ Created src/memory_layer.py


In [11]:
# Create agents directory
(PROJECT_ROOT / "src" / "agents").mkdir(exist_ok=True)

# __init__.py
(PROJECT_ROOT / "src" / "agents" / "__init__.py").write_text("from .boxing_coach import BoxingCoach\n")

# base_coach.py
base_coach_code = '''from typing import Optional, Dict
from ..memory_layer import MemoryBackedLLM

class BaseCoach:
    agent_id: str = "generic"
    
    def __init__(self, mem_llm: MemoryBackedLLM, system_prompt: str):
        self.mem_llm = mem_llm
        self.system_prompt = system_prompt
    
    def chat(self, user_id: str, message: str, context_data: Optional[Dict] = None) -> str:
        return self.mem_llm.chat(self.agent_id, user_id, self.system_prompt, message, context_data)
'''

(PROJECT_ROOT / "src" / "agents" / "base_coach.py").write_text(base_coach_code)

# boxing_coach.py
boxing_coach_code = '''from .base_coach import BaseCoach

BOXING_SYSTEM_PROMPT = """You are SAMMO Fight IQ, an elite boxing coach.

Fighter: Nolan, 47, 5'7" pressure fighter (Foreman/Pitbull Cruz style)
Goals: Sustainable pressure, guard discipline, smarter body work

Your style:
- SAFETY FIRST - flag defensive vulnerabilities immediately
- Short, concrete feedback (2-3 actionable cues)
- Use metrics when provided (danger_score, guard_down_ratio, etc.)
- Honest assessment, no fluff
- Connect flaws to consequences

When analyzing stats:
- danger_score > 0.6 = HIGH RISK
- guard_down_ratio > 30% = dangerous habit
- form_score < 0.7 = technique breakdown
- focus_next_round = training priority

Response format:
1. What the data shows (specific)
2. The immediate risk/opportunity
3. 1-2 concrete drills to fix it
4. Reference past sessions when relevant

You're building a fighter who can defend themselves safely."""

class BoxingCoach(BaseCoach):
    agent_id = "boxing"
    
    def __init__(self, mem_llm):
        super().__init__(mem_llm, system_prompt=BOXING_SYSTEM_PROMPT)
'''

(PROJECT_ROOT / "src" / "agents" / "boxing_coach.py").write_text(boxing_coach_code)
print("✅ Created boxing coach")

✅ Created boxing coach


In [12]:
# src/__init__.py
(PROJECT_ROOT / "src" / "__init__.py").write_text("")

print("✅ All core files created")
print(f"\n📂 Project structure:")
print(f"   {PROJECT_ROOT}")
print(f"   ├── src/")
print(f"   │   ├── simple_memory.py")
print(f"   │   ├── llm_client.py")
print(f"   │   ├── memory_layer.py")
print(f"   │   └── agents/")
print(f"   │       ├── base_coach.py")
print(f"   │       └── boxing_coach.py")
print(f"   ├── data/")
print(f"   └── mem_data/")

✅ All core files created

📂 Project structure:
   /opt/app-root/src/sammo-fight-iq
   ├── src/
   │   ├── simple_memory.py
   │   ├── llm_client.py
   │   ├── memory_layer.py
   │   └── agents/
   │       ├── base_coach.py
   │       └── boxing_coach.py
   ├── data/
   └── mem_data/


In [13]:
import sys
sys.path.insert(0, str(PROJECT_ROOT))

from src.simple_memory import SimpleMemoryStore

# Test memory store
mem_store = SimpleMemoryStore(str(PROJECT_ROOT / "mem_data" / "test.jsonl"))

# Store test data
mem_store.append("boxing", "nolan", "User: Test message\nCoach: Test response")

# Retrieve
history = mem_store.get_recent("boxing", "nolan", k=5)

print("✅ Memory store works!")
print(f"   Stored and retrieved {len(history)} entries")
print(f"   Last entry: {history[-1][:100]}...")

✅ Memory store works!
   Stored and retrieved 1 entries
   Last entry: User: Test message
Coach: Test response...


In [16]:
import pandas as pd
import json
from pathlib import Path

PROJECT_ROOT = Path("/opt/app-root/src/sammo-fight-iq")

# Files are in different locations - let's use the actual paths
CSV_PATH = Path("/opt/app-root/src/data/video_round_stats.csv")
JSON_PATH = Path("/opt/app-root/src/ai_stats/round1.json")

# Load round stats
try:
    round_stats = pd.read_csv(CSV_PATH)
    print("✅ Round stats loaded:")
    print(round_stats)
    print()
except FileNotFoundError:
    print(f"❌ {CSV_PATH} not found")
    round_stats = None

# Load round1.json
try:
    with open(JSON_PATH) as f:
        round1 = json.load(f)
    
    print("✅ Round 1 data:")
    for key, value in round1.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.3f}")
        else:
            print(f"   {key}: {value}")
except FileNotFoundError:
    print(f"❌ {JSON_PATH} not found")
    round1 = None

✅ Round stats loaded:
  round_id  total_frames  pose_frames  pose_coverage  guard_down_ratio  \
0   round1         14095         8520        0.60447          0.010329   

   avg_left_guard_height  avg_right_guard_height  avg_hip_rotation  \
0               0.031025                 0.03553           0.04255   

   avg_stance_width  avg_head_y  video_danger_score  video_form_score  \
0           0.04255    0.630725            0.164409          9.157296   

  video_focus_next_round  
0      pressure_and_body  

✅ Round 1 data:
   round_id: round1
   total_frames: 14095
   pose_frames: 8520
   pose_coverage: 0.604
   guard_down_ratio: 0.010
   avg_left_guard_height: 0.031
   avg_right_guard_height: 0.036
   avg_hip_rotation: 0.043
   avg_stance_width: 0.043
   avg_head_y: 0.631
   video_danger_score: 0.164
   video_form_score: 9.157
   video_focus_next_round: pressure_and_body


In [15]:
import os
from pathlib import Path

# Search for the files
print("🔍 Searching for data files...\n")

# Check common locations
search_paths = [
    Path("/opt/app-root/src/sammo-fight-iq"),
    Path("/opt/app-root/src"),
    Path.home(),
]

for search_path in search_paths:
    if search_path.exists():
        # Search for CSV
        csv_files = list(search_path.rglob("video_round_stats.csv"))
        json_files = list(search_path.rglob("round1.json"))
        
        if csv_files:
            print(f"✅ Found video_round_stats.csv:")
            for f in csv_files:
                print(f"   {f}")
        
        if json_files:
            print(f"✅ Found round1.json:")
            for f in json_files:
                print(f"   {f}")

# Also check current working directory
print(f"\n📍 Current directory: {Path.cwd()}")
print(f"📍 Files here: {list(Path.cwd().glob('*'))[:10]}")

🔍 Searching for data files...

✅ Found video_round_stats.csv:
   /opt/app-root/src/data/video_round_stats.csv
✅ Found round1.json:
   /opt/app-root/src/ai_stats/round1.json
✅ Found video_round_stats.csv:
   /opt/app-root/src/data/video_round_stats.csv
✅ Found round1.json:
   /opt/app-root/src/ai_stats/round1.json

📍 Current directory: /opt/app-root/src/sammo-fight-iq/notebooks
📍 Files here: [PosixPath('/opt/app-root/src/sammo-fight-iq/notebooks/.ipynb_checkpoints'), PosixPath('/opt/app-root/src/sammo-fight-iq/notebooks/01_pose_detection_test.ipynb'), PosixPath('/opt/app-root/src/sammo-fight-iq/notebooks/02_video_processing.ipynb'), PosixPath('/opt/app-root/src/sammo-fight-iq/notebooks/03_model_inference_test.ipynb'), PosixPath('/opt/app-root/src/sammo-fight-iq/notebooks/Untitled.ipynb')]


In [17]:
# Copy files to project data directory
import shutil

DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Copy CSV
shutil.copy(CSV_PATH, DATA_DIR / "video_round_stats.csv")
print(f"✅ Copied CSV to {DATA_DIR / 'video_round_stats.csv'}")

# Copy JSON
shutil.copy(JSON_PATH, DATA_DIR / "round1.json")
print(f"✅ Copied JSON to {DATA_DIR / 'round1.json'}")

# Now you can use the project paths
round_stats = pd.read_csv(DATA_DIR / "video_round_stats.csv")
with open(DATA_DIR / "round1.json") as f:
    round1 = json.load(f)

print("\n✅ All data files now in project directory")

✅ Copied CSV to /opt/app-root/src/sammo-fight-iq/data/video_round_stats.csv
✅ Copied JSON to /opt/app-root/src/sammo-fight-iq/data/round1.json

✅ All data files now in project directory


In [18]:
from src.llm_client import LLMConfig, LocalLLMClient
from src.memory_layer import MemoryBackedLLM
from src.agents import BoxingCoach

# Mock LLM that doesn't need a server
class MockLLMClient:
    """Mock LLM for testing without server."""
    
    def __init__(self, config):
        self.config = config
    
    def complete(self, messages):
        # Simple mock response based on context
        user_msg = messages[-1]["content"].lower()
        
        # Check if stats are in context
        has_stats = any("danger_score" in str(m.get("content", "")) for m in messages)
        
        if has_stats:
            return """Looking at your stats:

Your danger score of 0.65 is in the MODERATE-HIGH risk zone. The main issue I'm seeing is guard discipline - you're dropping it 38% of the time.

Your left guard is averaging 0.42 (should be 0.55+). When you throw combinations, you're leaving your head exposed.

Fix this with the 'punch-and-return' drill:
- 3 rounds on the heavy bag
- Every combo MUST end with hands back to face
- Focus on form, not power
- Video yourself and watch for guard drops

Hip rotation at 28° is weak - wider stance + resistance band work will help."""
        else:
            return """Hey! Ready to work. Upload a round or tell me what you want to focus on today.

Remember from last time - we were working on guard discipline and hip rotation."""

# Initialize with mock
llm_config = LLMConfig(base_url="mock", model="mock")
llm_client = MockLLMClient(llm_config)
memory_llm = MemoryBackedLLM(llm_client, mem_path=str(PROJECT_ROOT / "mem_data" / "mem_store.jsonl"))
boxing_coach = BoxingCoach(memory_llm)

print("✅ Boxing coach initialized (MOCK MODE)")

✅ Boxing coach initialized (MOCK MODE)


In [19]:
# Test 1: Simple greeting
response = boxing_coach.chat(
    user_id="nolan",
    message="Hey coach, ready to review my round?"
)

print("🥊 SAMMO:")
print(response)
print("\n" + "="*60 + "\n")

# Test 2: With round data (if available)
if round1:
    response = boxing_coach.chat(
        user_id="nolan",
        message="What did you see in my last round? What should I focus on?",
        context_data=round1
    )
    
    print("🥊 SAMMO (analyzing Round 1):")
    print(response)
    print("\n" + "="*60 + "\n")

# Test 3: Follow-up (tests memory)
response = boxing_coach.chat(
    user_id="nolan",
    message="Remind me what drill you said to do?"
)

print("🥊 SAMMO (follow-up):")
print(response)

🥊 SAMMO:
Looking at your stats:

Your danger score of 0.65 is in the MODERATE-HIGH risk zone. The main issue I'm seeing is guard discipline - you're dropping it 38% of the time.

Your left guard is averaging 0.42 (should be 0.55+). When you throw combinations, you're leaving your head exposed.

Fix this with the 'punch-and-return' drill:
- 3 rounds on the heavy bag
- Every combo MUST end with hands back to face
- Focus on form, not power
- Video yourself and watch for guard drops

Hip rotation at 28° is weak - wider stance + resistance band work will help.


🥊 SAMMO (analyzing Round 1):
Looking at your stats:

Your danger score of 0.65 is in the MODERATE-HIGH risk zone. The main issue I'm seeing is guard discipline - you're dropping it 38% of the time.

Your left guard is averaging 0.42 (should be 0.55+). When you throw combinations, you're leaving your head exposed.

Fix this with the 'punch-and-return' drill:
- 3 rounds on the heavy bag
- Every combo MUST end with hands back to face
